In [1]:
import requests
import pandas as pd
import numpy as np
from lxml import html
from lxml import etree

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}

url_army = 'https://www.anekdot.ru/tags/%D0%B0%D1%80%D0%BC%D0%B8%D1%8F/'
url_money = 'https://www.anekdot.ru/tags/%D0%B4%D0%B5%D0%BD%D1%8C%D0%B3%D0%B8/'
url_school = 'https://www.anekdot.ru/tags/%D1%88%D0%BA%D0%BE%D0%BB%D0%B0/'

text_army = []
text_money = []
text_school = []

In [3]:
"""собираем данные с сайта, сохраняем в листы по типам"""

for i in range(1, 22):
    r_army = requests.get(url_army + str(i), headers = headers)
    
    if r_army.status_code == 200:    
        tree = html.fromstring(r_army.content)
        film_list_lxml = tree.xpath('.//div[@class = "topicbox"]/div[@class = "text"]')
        
        for h in film_list_lxml:
            text = h.xpath("string()")
            text_army.append(text)
    else:
        print('Unable to get responce :(')
        
        
        
for i in range(1, 23):
    r_money = requests.get(url_money + str(i), headers = headers)
    
    if r_money.status_code == 200:    
        tree = html.fromstring(r_money.content)
        film_list_lxml = tree.xpath('.//div[@class = "topicbox"]/div[@class = "text"]')
        
        for h in film_list_lxml:
            text = h.xpath("string()")
            text_money.append(text)
    else:
        print('Unable to get responce :(')
        
        
        
for i in range(1, 26):
    r_school = requests.get(url_school + str(i), headers = headers)
    
    if r_school.status_code == 200:    
        tree = html.fromstring(r_school.content)
        film_list_lxml = tree.xpath('.//div[@class = "topicbox"]/div[@class = "text"]')
        
        for h in film_list_lxml:
            text = h.xpath("string()")
            text_school.append(text)
    else:
        print('Unable to get responce :(')

Получаем 3 листа по тематике, элементы которых - анекдоты.

In [4]:
"""запихиваем в один датафрэйм"""

dataset = pd.DataFrame(columns=['text', 'label'])

data_frame = pd.DataFrame(columns=['text', 'label'])
data_frame['text'] = text_army
data_frame['label'] = 0         #army

dataset = dataset.append(data_frame, ignore_index=True, sort=False)

data_frame = pd.DataFrame(columns=['text', 'label'])
data_frame['text'] = text_money
data_frame['label'] = 1         #money

dataset = dataset.append(data_frame, ignore_index=True, sort=False)

data_frame = pd.DataFrame(columns=['text', 'label'])
data_frame['text'] = text_school
data_frame['label'] = 2         #school

dataset = dataset.append(data_frame, ignore_index=True, sort=False)

In [5]:
"""удаляем повторы и перемешиваем"""
dataset.drop_duplicates(inplace=True)
dataset = dataset.sample(frac=1).reset_index(drop=True) #mix

In [6]:
"""сохраняем в файл"""
dataset.to_csv('./jokes.csv', index=False)

Получаем датасет следующего вида: (текс анекдота, метка класса). 0 - армия, 1 - деньги, 2 - школа.

In [7]:
#dataset = pd.read_csv('./jokes.csv')

In [8]:
dataset.tail(15)

,text,label
3258,Ещё одна армейская история. Если кто не понял ...,0
3259,"После успеха передачи на НТВ ""Поедем, поедим"",...",1
3260,РВСН. Авария.Зона обеспечения была у нас в осн...,0
3261,"Из школьного сочинения: Динозавры вымерли, так...",2
3262,Сын подрабатывает преподавателем в институте н...,1
3263,Прочитал только что не могу не поделиться.Посл...,0
3264,Российская экономика напоминает тетрис: деньги...,1
3265,Вдогонку истории про преступление с колбасой в...,0
3266,"Бабушка спросила у внука (первоклассника), что...",2
3267,Классная руководительница – родителям:- Поздра...,2


In [9]:
dataset['label'].value_counts()

2    1201
1    1077
0     995
Name: label, dtype: int64

Нет явного дисбаланса классов, для оценки качества подойдет метрика accuracy.

In [10]:
"""подготавливаем данные, разбиваем на обучающую и тестовую"""
from sklearn.model_selection import train_test_split


texts = dataset['text'].values
target = dataset['label'].values.astype(int)

texts_train, texts_test, y_train, y_test = train_test_split(texts, target, test_size=0.3, random_state=42)

На тест оставляем 30%.

In [11]:
"""Производим векторизацию (tf-idf), лимит словаря - 10 000"""
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_features = 10000)
vectorizer.fit(texts_train)

X_train = vectorizer.transform(texts_train)
X_test = vectorizer.transform(texts_test)

Переводим документы в векторы.

Наша модель - логистическая регрессия, подбираем лучшие параметры (коэффициент регуляризации и метрику) 5 кратная кросс-валидация.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

parameters = {'C': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 500, 1000, 2000], 'penalty' : ['l1', 'l2']}

clf = RandomizedSearchCV(LogisticRegression(), parameters, cv=5, n_jobs=-1)

clf.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 500, 1000, 2000], 'penalty': ['l1', 'l2']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [13]:
clf.best_params_

{'penalty': 'l2', 'C': 50}

Создаем и обучаем классификатор с определенными лучшими параметрами.

In [14]:
tfidf_model = LogisticRegression(C=50, penalty='l2')
tfidf_model.fit(X_train, y_train)

print('Train score =', tfidf_model.score(X_train, y_train))
print('Test score  =', tfidf_model.score(X_test, y_test))

Train score = 0.9930161501527717
Test score  = 0.8940936863543788


На обучающей выборке точность составляет 99.3%, на тестовой - 89.4%.